# 4. Building a Multi-Layer Neural Network

# IMPORT LIBRARIES

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from keras.models import Sequential 
from keras.layers import Dense

from keras.callbacks import EarlyStopping,ModelCheckpoint 
from tensorflow.keras.optimizers import Adam  # type: ignore
from sklearn.preprocessing import StandardScaler

SEED =2017

# Load Data

In [20]:
data = pd.read_csv("winequality-red.csv" , sep=",")
y = data["quality"]

In [21]:
X = data.drop(['quality'], axis=1)  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)  
# Print average quality and first rows of training set  
print('Average quality training set: {:.4f}'.format(y_train.mean())) 
X_train.head()  
scaler = StandardScaler().fit(X_train)  
X_train = pd.DataFrame(scaler.transform(X_train)) 
X_test = pd.DataFrame(scaler.transform(X_test)) 
print('MSE:', np.mean((y_test - ([y_train.mean()] * y_test.shape[0])) ** 2)) 
print('MSE:', np.mean((y_test - ([y_train.mean()] * 
y_test.shape[0])) ** 2))  

# Now, let's build our neural network by defining the network architecture 
model = Sequential()  
# First hidden layer with 100 hidden units  
model.add(Dense(200, input_dim=X_train.shape[1], activation='relu'))   
# Second hidden layer with 50 hidden units 
model.add(Dense(25, activation='relu'))  
# Output layer 
model.add(Dense(1, 
activation='linear'))  
# Set optimizer opt = Adam() # Compile model  


Average quality training set: 5.6231
MSE: 0.5939855630834537
MSE: 0.5939855630834537


c:\Users\heman\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Define the optimizer
optimizer = Adam()  # You can customize the optimizer with parameters if needed

# Compile the model with the defined optimizer
model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])  # Use 'mae' (mean absolute error) as a metric

# Define the callbacks for early stopping and model checkpointing
callbacks = [
    EarlyStopping(monitor='val_loss', patience=30, verbose=2),
    ModelCheckpoint('checkpoints/multi_layer_best_model.keras', monitor='val_loss', save_best_only=True, verbose=0)
]

batch_size = 64
n_epochs = 5000

# Train the model
model.fit(X_train.values, y_train, batch_size=batch_size, epochs=n_epochs, validation_split=0.2, verbose=2,
          validation_data=(X_test.values, y_test), callbacks=callbacks)

# Print model summary
model.summary()

# Load the best model (including weights and optimizer state)
best_model = tf.keras.models.load_model('checkpoints/multi_layer_best_model.keras')

# Evaluate the model on the test set
score = best_model.evaluate(X_test.values, y_test, verbose=0)
print('Test mean absolute error: %.2f' % score[1])  # Print the MAE instead of accuracy


Epoch 1/5000
20/20 - 1s - 65ms/step - loss: 24.5202 - mae: 4.8186 - val_loss: 16.1738 - val_mae: 3.8911
Epoch 2/5000
20/20 - 0s - 7ms/step - loss: 9.2778 - mae: 2.7763 - val_loss: 5.1741 - val_mae: 2.0173
Epoch 3/5000
20/20 - 0s - 7ms/step - loss: 3.3964 - mae: 1.4798 - val_loss: 2.8323 - val_mae: 1.3273
Epoch 4/5000
20/20 - 0s - 7ms/step - loss: 2.4847 - mae: 1.2362 - val_loss: 2.3618 - val_mae: 1.2336
Epoch 5/5000
20/20 - 0s - 8ms/step - loss: 2.0410 - mae: 1.1392 - val_loss: 2.1107 - val_mae: 1.1586
Epoch 6/5000
20/20 - 0s - 7ms/step - loss: 1.8321 - mae: 1.0764 - val_loss: 1.9331 - val_mae: 1.1025
Epoch 7/5000
20/20 - 0s - 7ms/step - loss: 1.6748 - mae: 1.0256 - val_loss: 1.7823 - val_mae: 1.0561
Epoch 8/5000
20/20 - 0s - 6ms/step - loss: 1.5490 - mae: 0.9883 - val_loss: 1.6759 - val_mae: 1.0232
Epoch 9/5000
20/20 - 0s - 6ms/step - loss: 1.4281 - mae: 0.9467 - val_loss: 1.5635 - val_mae: 0.9853
Epoch 10/5000
20/20 - 0s - 7ms/step - loss: 1.3321 - mae: 0.9153 - val_loss: 1.4811 - va

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 200)            │         2,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 25)             │         5,025 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,355 (87.33 KB)

 Trainable params: 7,451 (29.11 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 14,904 (58.22 KB)

Test mean absolute error: 0.45
